# Track-cluster-matching 

So here are the results from running the the `analysis.sub` file in `src` but using `create_jet_based_tree.py` with the 3rd argument `True` that will then artificially do track-cluster matching for neutral pfos which have an charged MC. If there is any track associated to the MC I will "add" it to the pfo aka modifying the `type` and adding track information (like SIP, cov matrix..). 

In [1]:
save_status = False

In [2]:
import uproot 
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [3]:
def load_data(file_name):
    file = uproot.open(file_name)
    tree = file['tree;1']
    data = tree.arrays(library="np", entry_start=0, entry_stop=25000) #entry_stop=1000000)
    return data

In [4]:
path= "/eos/experiment/fcc/ee/datasets/CLD_fullsim_tagging_input_artif-track-clusster-matching/hadded"
dic = {
    "uu": np.array([]),
    "dd": np.array([]),
    "ss": np.array([]),
    "cc": np.array([]),
    "bb": np.array([]),
    "gg": np.array([]),
    "tautau": np.array([])
}

for process in dic.keys():
    data = load_data(f"{path}/H{process}.root")
    dic[process] = data

In [5]:
dic["bb"].keys()

dict_keys(['event_number', 'n_hit', 'n_part', 'jet_p', 'jet_e', 'jet_mass', 'jet_nconst', 'jet_npfcand', 'jet_theta', 'jet_phi', 'recojet_isG', 'recojet_isU', 'recojet_isD', 'recojet_isS', 'recojet_isC', 'recojet_isB', 'recojet_isTAU', 'pfcand_e', 'pfcand_p', 'pfcand_theta', 'pfcand_phi', 'pfcand_type', 'pfcand_charge', 'pfcand_isEl', 'pfcand_isMu', 'pfcand_isGamma', 'pfcand_isNeutralHad', 'pfcand_isChargedHad', 'jet_nmu', 'jet_nel', 'jet_ngamma', 'jet_nnhad', 'jet_nchad', 'pfcand_erel_log', 'pfcand_phirel', 'pfcand_thetarel', 'pfcand_dptdpt', 'pfcand_detadeta', 'pfcand_dphidphi', 'pfcand_dxydxy', 'pfcand_dzdz', 'pfcand_dxydz', 'pfcand_dphidxy', 'pfcand_dlambdadz', 'pfcand_dxyc', 'pfcand_dxyctgtheta', 'pfcand_phic', 'pfcand_phidz', 'pfcand_phictgtheta', 'pfcand_cdz', 'pfcand_cctgtheta', 'pfcand_dxy', 'pfcand_dz', 'pfcand_btagSip2dVal', 'pfcand_btagSip2dSig', 'pfcand_btagSip3dVal', 'pfcand_btagSip3dSig', 'pfcand_btagJetDistVal', 'pfcand_btagJetDistSig', 'pfcand_mtof', 'pfcand_dndx', 'pf

Now lets consider the plots - what do I need to do? 


so I have the key `pfcand_track_cluster_matching` which is either 
- 0: no correction (neutral)
- 1: correction (neutral)
- 2: couldn't apply correction (neutral)
- 3: charged (not applicable)


So, lets look at all particles that are not 3. And then plots 0,1,2, as a fraction of all over momentum and theta for different H channels. 

In [6]:
def return_i_neutrals(Htype, i, obs):
    """find partilces that have either 0,1,2 for pfcand_track_cluster_matching. Obs could be theta or momentum"""
    index = np.where(np.concatenate(dic[Htype]["pfcand_track_cluster_matching"]) == i)
    return np.concatenate(dic[Htype][obs])[index]

In [7]:
# Normalize function
def normalize_hist(data, ref_counts, bins, range):
    counts, _ = np.histogram(data, bins=bins, range=range, density=False)
    # Avoid division by zero by using np.where
    normalized_counts = np.where(ref_counts > 0, counts / ref_counts, 0)
    return normalized_counts

In [ ]:
def plot_matching(Htype, save=False):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))

    ref_counts, bin_edges = np.histogram(np.concatenate(dic[Htype][obs])[i_ptype[i]], range=ranges[j], bins=bins, density=False)
    # plot
            obs_data = np.concatenate(dic[Htype][obs])[ind_interest]
            normalized_obs_data = normalize_hist(obs_data, ref_counts, bins, range=ranges[j])
            bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # Calculate bin centers for plotting
    
    if save:
        plt.savefig(f"./hist-matching/matching_{Htype}.pdf")
    plt.show()